In [1]:
import transformers
import torch
from transformers import BertForSequenceClassification

root_path = "/home/mai.kassem/Documents/Med-BERT/"
BertForSequenceClassification.from_pretrained(root_path + "MedBERT Pretraining/")

/home/mai.kassem/.conda/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
-

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(82603, 192, padding_idx=0)
      (position_embeddings): Embedding(512, 192)
      (token_type_embeddings): Embedding(1000, 192)
      (LayerNorm): LayerNorm((192,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=192, out_features=192, bias=True)
              (key): Linear(in_features=192, out_features=192, bias=True)
              (value): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=192, out_features=192, bias=True)
              (LayerNorm): LayerNorm((192,), eps=1e-12, elem

In [2]:
torch.load(root_path + "MedBERT Pretraining/pytorch_model.bin")

OrderedDict([('bert.embeddings.position_ids',
              tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                       1

In [3]:
# Required Packages
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from transformers import BertForSequenceClassification
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import matplotlib
import transformers
import time
import tqdm
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
%matplotlib inline
use_cuda = torch.cuda.is_available()


In [4]:
train_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.train",
        "rb",
    ),
    encoding="bytes",
)
valid_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.valid",
        "rb",
    ),
    encoding="bytes",
)
test_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.test",
        "rb",
    ),
    encoding="bytes",
)

In [5]:
print("Length of training dataset:", len(train_f))
print("length of validation dataset", len(valid_f))
print("length of testing dataset", len(test_f))

Length of training dataset: 12038
length of validation dataset 1719
length of testing dataset 3439


In [6]:
train_f[0]

[11046564,
 1,
 [8931, 261, 1031, 192, 4026, 205, 11799, 397, 2391, 3107],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [7]:
# The version in this file is updated to dump pt_id for better visualization and results analysis

# Below are key functions for  Data prepartion ,formating input data into features, and model defintion
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    Based on original BERT code: We use this class instead of `None` because treating `None` as padding
    batches could cause silent errors.
    """


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(
        self, input_ids, input_mask, segment_ids, label_id, pt_id, is_real_example=True
    ):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.is_real_example = is_real_example
        self.pt_id = pt_id


def convert_EHRexamples_to_features(examples, max_seq_length):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        feature = convert_singleEHR_example(ex_index, example, max_seq_length)
        features.append(feature)
    return features


# This is the EHR version
def convert_singleEHR_example(ex_index, example, max_seq_length):
    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_id=0,
            is_real_example=False,
        )

    input_ids = example[2]
    segment_ids = example[3]
    label_id = example[1]
    pt_id = example[0]

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # LR 5/13 Left Truncate longer sequence
    while len(input_ids) > max_seq_length:
        input_ids = input_ids[-max_seq_length:]
        input_mask = input_mask[-max_seq_length:]
        segment_ids = segment_ids[-max_seq_length:]

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    # TODO remove label_id
    # feature = [input_ids, input_mask, segment_ids, label_id, pt_id, True]
    # feature = [input_ids, input_mask, segment_ids, pt_id, True]
    feature = {"input_ids": input_ids, "input_mask": input_mask, "segment_ids":
    segment_ids, "pt_id": pt_id, "a": True}
    return feature

In [8]:
class BERTdataEHR(Dataset):
    # TODO add labels
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.features.items()}
        item = {key: torch.tensor(val) for key, val in self.features[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
MAX_SEQ_LENGTH = 75

train_labels = []
for (ex_index, example) in enumerate(train_f):
    label = example[1]
    train_labels.append(label)
test_labels = []
for (ex_index, example) in enumerate(test_f):
    label = example[1]
    test_labels.append(label)
valid_labels = []
for (ex_index, example) in enumerate(valid_f):
    label = example[1]
    valid_labels.append(label)

# Data Preparation
train_features = convert_EHRexamples_to_features(train_f, MAX_SEQ_LENGTH)
test_features = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
valid_features = convert_EHRexamples_to_features(valid_f, MAX_SEQ_LENGTH)
train_dataset = BERTdataEHR(train_features, train_labels)
test_dataset = BERTdataEHR(test_features, test_labels)
valid_dataset = BERTdataEHR(valid_features, valid_labels)

In [10]:
train_dataset

In [11]:
# Hyperparameter tunning with Optuna.

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import json

from transformers import EvalPrediction
import numpy as np

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"AUC":roc_auc_score(p.label_ids, preds)}
    # return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

training_args = TrainingArguments(
    output_dir='Gentacimin_finetuning',    # output directory
    #evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch.
    evaluation_strategy="steps",      # Evaluation is done (and logged) every eval_steps.
    eval_steps=1000,                  # Number of update steps between two evaluations 
    per_device_eval_batch_size=64,    # batch size for evaluation
    save_total_limit=1,               # limit the total amount of checkpoints. Deletes the older checkpoints.
)

def model_init():
    model = BertForSequenceClassification.from_pretrained(root_path + "MedBERT Pretraining/")
    for param in model.base_model.parameters():
        param.requires_grad = False
    return model

trainer = Trainer(
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # metrics to be computed
    model_init=model_init                # Instantiate model before training starts
)

def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 2e-3, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 100),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
    }
def my_objective(metrics):
    return metrics["eval_loss"]

best_run = trainer.hyperparameter_search(direction="minimize", hp_space=my_hp_space, compute_objective=my_objective, n_trials=100)

# with open("/content/drive/My Drive/cord19/best_run.json", "w+") as f:
#   f.write(json.dumps(best_run.hyperparameters))

loading configuration file /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 192,
  "initializer_range": 0.02,
  "intermediate_size": 64,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 6,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1000,
  "use_cache": true,
  "vocab_size": 82603
}

loading weights file /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/pytorch_model.bin
Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_rel

Step,Training Loss,Validation Loss,Auc
1000,0.311800,0.290593,0.499373
2000,0.326000,0.353320,0.499373
3000,0.318200,0.310189,0.499373
4000,0.298800,0.314019,0.499373
5000,0.336900,0.329224,0.499373
6000,0.320600,0.262593,0.500000
7000,0.312400,0.336501,0.499373
8000,0.336800,0.351497,0.499373
9000,0.351000,0.274063,0.499686
10000,0.303900,0.292766,0.499686


Saving model checkpoint to Gentacimin_finetuning/run-0/checkpoint-500
Configuration saved in Gentacimin_finetuning/run-0/checkpoint-500/config.json
Model weights saved in Gentacimin_finetuning/run-0/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1719
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: a, segment_ids, pt_id, input_mask. If a, segment_ids, pt_id, input_mask are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to Gentacimin_finetuning/run-0/checkpoint-1000
Configuration saved in Gentacimin_finetuning/run-0/checkpoint-1000/config.json
Model weights saved in Gentacimin_finetuning/run-0/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Gentacimin_finetuning/run-0/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to Gentacimin_finetuni

Step,Training Loss,Validation Loss,Auc
1000,0.286300,0.268349,0.499373
2000,0.278100,0.270557,0.499373
3000,0.279700,0.283066,0.499686
4000,0.278000,0.267615,0.499373
5000,0.280900,0.262584,0.499373
6000,0.280400,0.261916,0.499686
7000,0.276200,0.263832,0.499373
8000,0.262000,0.276920,0.499373
9000,0.272400,0.276631,0.499373
10000,0.288800,0.262623,0.499686


Saving model checkpoint to Gentacimin_finetuning/run-1/checkpoint-500
Configuration saved in Gentacimin_finetuning/run-1/checkpoint-500/config.json
Model weights saved in Gentacimin_finetuning/run-1/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1719
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: a, segment_ids, pt_id, input_mask. If a, segment_ids, pt_id, input_mask are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to Gentacimin_finetuning/run-1/checkpoint-1000
Configuration saved in Gentacimin_finetuning/run-1/checkpoint-1000/config.json
Model weights saved in Gentacimin_finetuning/run-1/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [Gentacimin_finetuning/run-1/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to Gentacimin_finetuni

...

Saving model checkpoint to Gentacimin_finetuning/run-97/checkpoint-500
Configuration saved in Gentacimin_finetuning/run-97/checkpoint-500/config.json
Model weights saved in Gentacimin_finetuning/run-97/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1719
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: a, segment_ids, pt_id, input_mask. If a, segment_ids, pt_id, input_mask are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[I 2022-12-16 03:49:26,112] Trial 97 pruned. 
Trial: {'learning_rate': 0.0014149294579052008, 'num_train_epochs': 98, 'seed': 1, 'per_device_train_batch_size': 32}
loading configuration file /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",


Step,Training Loss,Validation Loss,Auc
1000,0.268600,0.265233,0.499373


Saving model checkpoint to Gentacimin_finetuning/run-98/checkpoint-500
Configuration saved in Gentacimin_finetuning/run-98/checkpoint-500/config.json
Model weights saved in Gentacimin_finetuning/run-98/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1719
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: a, segment_ids, pt_id, input_mask. If a, segment_ids, pt_id, input_mask are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[I 2022-12-16 03:49:36,367] Trial 98 pruned. 
Trial: {'learning_rate': 0.0017739331217471893, 'num_train_epochs': 85, 'seed': 8, 'per_device_train_batch_size': 32}
loading configuration file /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",


Step,Training Loss,Validation Loss,Auc
1000,0.271500,0.263430,0.499373


Saving model checkpoint to Gentacimin_finetuning/run-99/checkpoint-500
Configuration saved in Gentacimin_finetuning/run-99/checkpoint-500/config.json
Model weights saved in Gentacimin_finetuning/run-99/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1719
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: a, segment_ids, pt_id, input_mask. If a, segment_ids, pt_id, input_mask are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[I 2022-12-16 03:49:46,673] Trial 99 pruned. 


In [12]:
best_run.hyperparameters

{'learning_rate': 0.0018650058619830382,
 'num_train_epochs': 91,
 'seed': 5,
 'per_device_train_batch_size': 32}